## Preprocessing the data

In [1]:
import pandas as pd

In [90]:
data = pd.read_csv("data/SpotifyFeatures.csv")

In [91]:
data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [92]:
popularity_class = []

for index, row in data.iterrows():
    if row['popularity'] > 50:
        popularity_class.append(1)
    else:
        popularity_class.append(0)

In [93]:
from collections import Counter
Counter(popularity_class)

Counter({0: 153123, 1: 79602})

In [94]:
data['popularity_class'] = popularity_class

In [95]:
data.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence',
       'popularity_class'],
      dtype='object')

In [96]:
from sklearn.preprocessing import StandardScaler

In [97]:
sc = StandardScaler()
data['duration_ms'] = sc.fit_transform(data[['duration_ms']])
data['loudness'] = sc.fit_transform(data[['loudness']])
data['tempo'] = sc.fit_transform(data[['tempo']])

In [10]:
#data = pd.get_dummies(data, columns = ["key", "mode"])

In [98]:
data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity_class
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,-1.141368,0.910,0.000,C#,0.3460,1.290703,Major,0.0525,1.595607,4/4,0.814,0
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,-0.821867,0.737,0.000,F#,0.1510,0.668683,Minor,0.0868,1.823253,4/4,0.816,0
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,-0.545298,0.131,0.000,C,0.1030,-0.718402,Minor,0.0362,-0.588326,5/4,0.368,0
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,-0.695295,0.326,0.000,C#,0.0985,-0.434817,Major,0.0395,1.750597,4/4,0.227,0
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,-1.282184,0.225,0.123,F,0.2020,-1.930601,Major,0.0456,0.741433,4/4,0.390,0


In [99]:
data.shape

(232725, 19)

In [100]:
data = data.drop_duplicates(subset=['artist_name', 'track_name'])

In [101]:
data.shape

(176514, 19)

In [102]:
data.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence',
       'popularity_class'],
      dtype='object')

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
X = data.drop(['genre', 'artist_name', 'track_name', 'track_id', 'time_signature', 'popularity', 'popularity_class', 'key', 'mode'], axis=1)

In [105]:
X_train = X.values[:130000]
y_train = data['popularity_class'][:130000]
X_test = X.values[130000:]
y_test = data['popularity_class'][130000:]

In [106]:
Counter(y_train)

Counter({0: 96572, 1: 33428})

In [107]:
#trying to balance, this is kind of weird rn, it makes the results on the test set worse, but better for top2017
positive_cases = data[:130000][data[:130000]['popularity_class'] == 1]
negative_sample = data[:130000][data[:130000]['popularity_class'] == 0].sample(33428, random_state=0)
train_balanced = pd.concat([positive_cases, negative_sample])

In [108]:
X = train_balanced.drop(['genre', 'artist_name', 'track_name', 'track_id', 'time_signature', 'popularity', 'popularity_class', 'key', 'mode'], axis=1)

In [109]:
X_train = X.values
y_train = train_balanced['popularity_class']

In [71]:
#X_train, X_test, y_train, y_test = train_test_split(X.values, data['popularity_class'], test_size=0.2, random_state=0)

## Model training and initial testing

In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [124]:
rf = RandomForestClassifier(n_estimators=100)

In [113]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [114]:
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                     weights='uniform')

In [115]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [116]:
from sklearn.metrics import accuracy_score

In [117]:
y_pred_rf = rf.predict(X_test)
y_pred_knn = knn.predict(X_test)
y_pred_dtc = dtc.predict(X_test)

print("random forest accuracy", accuracy_score(y_test, y_pred_rf))
print("k nearest neighbors accuracy", accuracy_score(y_test, y_pred_knn))
print("decision tree accuracy", accuracy_score(y_test, y_pred_dtc))

random forest accuracy 0.7095283140559832
k nearest neighbors accuracy 0.635550586920067
decision tree accuracy 0.641376789783721


## Cross-validation

In [125]:
from sklearn.model_selection import cross_validate

In [138]:
X = data.drop(['genre', 'artist_name', 'track_name', 'track_id', 'time_signature', 'popularity', 'popularity_class', 'key', 'mode'], axis=1)

In [140]:
rf = RandomForestClassifier(n_estimators=100)
cv_results_rf = cross_validate(rf, X, data['popularity_class'], cv=5, return_train_score=True)

In [141]:
print("random forest cross-validation")
print(cv_results_rf["test_score"])
print(cv_results_rf["train_score"])
print(cv_results_rf["test_score"].mean())

random forest cross-validation
[0.7251303  0.79398935 0.79193813 0.77890771 0.78049402]
[0.99963175 0.99925643 0.99916438 0.99932725 0.99920687]
0.7740919027857175


In [142]:
knn = KNeighborsClassifier(n_neighbors=100)
cv_results_knn = cross_validate(knn, X, data['popularity_class'], cv=5, return_train_score=True)

In [143]:
print("k nearest neighbors cross-validation")
print(cv_results_knn["test_score"])
print(cv_results_knn["train_score"])
print(cv_results_knn["test_score"].mean())

k nearest neighbors cross-validation
[0.74764899 0.7903637  0.79593224 0.78536627 0.78530961]
[0.80293889 0.79808795 0.79370025 0.79685154 0.80066142]
0.7809241627931586


In [144]:
dtc = DecisionTreeClassifier()
cv_results_dtc = cross_validate(dtc, X, data['popularity_class'], cv=5, return_train_score=True)

In [145]:
print("decision tree cross-validation")
print(cv_results_dtc["test_score"])
print(cv_results_dtc["train_score"])
print(cv_results_dtc["test_score"].mean())

decision tree cross-validation
[0.61701224 0.70300816 0.68664665 0.70613563 0.71296244]
[0.99963884 0.99926351 0.99917146 0.99934142 0.99922103]
0.6851530222605599


## Testing on 2017 top

In [152]:
top2017 = pd.read_csv("data/featuresdf2017.csv")

In [153]:
popularity_class = [1]*100
top2017['popularity_class'] = popularity_class

top2017['duration_ms'] = sc.fit_transform(top2017[['duration_ms']])
top2017['loudness'] = sc.fit_transform(top2017[['loudness']])
top2017['tempo'] = sc.fit_transform(top2017[['tempo']])

#top2017 = pd.get_dummies(top2017, columns = ["key", "mode"])

In [154]:
top2017.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity_class
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,1.377358,0.0,0.0802,0.5810,0.000000,0.0931,0.931,-0.835063,0.468871,4.0,1
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,0.738776,1.0,0.1200,0.2290,0.000000,0.0924,0.813,-1.088400,0.319390,4.0,1
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,0.499516,1.0,0.1700,0.2090,0.000000,0.1120,0.846,2.108041,0.300208,4.0,1
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-0.622604,0.0,0.0317,0.0498,0.000014,0.1640,0.446,-0.581871,0.880266,4.0,1
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,0.763315,1.0,0.0367,0.0552,0.000000,0.1670,0.811,-1.376289,2.148071,4.0,1


In [155]:
X_top2017 = top2017.drop(['id', 'name', 'artists', 'time_signature', 'popularity_class', 'key', 'mode'], axis=1)
"""X_top2017.columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'key_A', 'key_A#', 'key_B', 'key_C', 'key_C#', 'key_D',
       'key_D#', 'key_E', 'key_F', 'key_F#', 'key_G', 'key_G#', 'mode_Major',
       'mode_Minor']"""

"X_top2017.columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',\n       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',\n       'key_A', 'key_A#', 'key_B', 'key_C', 'key_C#', 'key_D',\n       'key_D#', 'key_E', 'key_F', 'key_F#', 'key_G', 'key_G#', 'mode_Major',\n       'mode_Minor']"

In [156]:
X_top2017.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.825,0.652,1.377358,0.0802,0.5810,0.000000,0.0931,0.931,-0.835063,0.468871
1,0.694,0.815,0.738776,0.1200,0.2290,0.000000,0.0924,0.813,-1.088400,0.319390
2,0.660,0.786,0.499516,0.1700,0.2090,0.000000,0.1120,0.846,2.108041,0.300208
3,0.617,0.635,-0.622604,0.0317,0.0498,0.000014,0.1640,0.446,-0.581871,0.880266
4,0.609,0.668,0.763315,0.0367,0.0552,0.000000,0.1670,0.811,-1.376289,2.148071


still fitting on X_train and y_train because those are balanced

In [168]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [169]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                     weights='uniform')

In [170]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [171]:
y_pred_2017_rf = rf.predict(X_top2017)
y_pred_2017_knn = knn.predict(X_top2017)
y_pred_2017_dtc = dtc.predict(X_top2017)

print("random forest accuracy (2017 top)", accuracy_score(top2017['popularity_class'], y_pred_2017_rf))
print("k nearest neighbors accuracy (2017 top)", accuracy_score(top2017['popularity_class'], y_pred_2017_knn))
print("decision tree accuracy (2017 top)", accuracy_score(top2017['popularity_class'], y_pred_2017_dtc))

random forest accuracy (2017 top) 0.34
k nearest neighbors accuracy (2017 top) 0.71
decision tree accuracy (2017 top) 0.31
